In [25]:
import cv2
import numpy as np
import time
from collections import defaultdict

# Path to your video file
video_path = "PXL_20250325_043922504.TS.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error opening video file")
    exit()

# For FPS calculation
prev_time = time.time()
frame_count = 0
fps = 0

def is_dashed(lines, threshold=20):
    if len(lines) < 4:
        return False
    lines = sorted(lines, key=lambda p: p[1])  # sort by y
    gaps = []
    for i in range(0, len(lines) - 2, 2):
        y1 = lines[i][1]
        y2 = lines[i + 2][1]
        gaps.append(abs(y2 - y1))
    if len(gaps) == 0:
        return False
    avg_gap = sum(gaps) / len(gaps)
    return avg_gap > threshold


# Corrected the __init__ method (previously it was _init_)
class LaneStateMachine:
    def __init__(self, stability_threshold=5):
        self.state = "No Lane Detected"
        self.counters = defaultdict(int)
        self.threshold = stability_threshold

    def update(self, new_state):
        for key in self.counters:
            if key != new_state:
                self.counters[key] = 0

        self.counters[new_state] += 1

        # If new_state is consistent for N frames, update state
        if self.counters[new_state] >= self.threshold:
            if new_state != self.state:
                self.state = new_state
                # Reset all counters after transition
                for key in self.counters:
                    self.counters[key] = 0
        return self.state


lane_state_machine = LaneStateMachine(stability_threshold=5)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (960, 540))
    height, width, _ = frame.shape

    # ---------------------------------------------------------------
    # Lane Detection ROI (lower half)
    lower_half = frame[height // 2:, :]
    crop_margin = int(0.15 * width)
    lower_half_cropped = lower_half[:, crop_margin:width - crop_margin]

    lab = cv2.cvtColor(lower_half_cropped, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    lab = cv2.merge((cl, a, b))
    lower_half_cropped = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    hsv_lane = cv2.cvtColor(lower_half_cropped, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([15, 30, 30])
    upper_yellow = np.array([40, 255, 255])
    lane_mask = cv2.inRange(hsv_lane, lower_yellow, upper_yellow)

    kernel = np.ones((5, 5), np.uint8)
    lane_mask = cv2.morphologyEx(lane_mask, cv2.MORPH_CLOSE, kernel)
    lane_mask = cv2.morphologyEx(lane_mask, cv2.MORPH_OPEN, kernel)

    yellow_segment = cv2.bitwise_and(lower_half_cropped, lower_half_cropped, mask=lane_mask)
    gray = cv2.cvtColor(yellow_segment, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 100, 200)

    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=30)
    line_image = np.zeros_like(frame)
    left_lines, right_lines = [], []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            x1 += crop_margin
            x2 += crop_margin
            y1 += height // 2
            y2 += height // 2

            if x2 - x1 == 0:
                continue

            slope = (y2 - y1) / (x2 - x1)

            if abs(slope) < 0.5:
                continue

            if slope < 0:
                left_lines.extend([(x1, y1), (x2, y2)])
            else:
                right_lines.extend([(x1, y1), (x2, y2)])

    left_dashed = is_dashed(left_lines)
    right_dashed = is_dashed(right_lines)

    if left_lines or right_lines:
        plot_y = np.linspace(height // 2, height - 1, num=height // 2)

        if left_lines:
            left_points = np.array(left_lines)
            left_fit = np.polyfit(left_points[:, 1], left_points[:, 0], 1)
            left_x = left_fit[0] * plot_y + left_fit[1]
            for x, y in zip(left_x, plot_y):
                cv2.circle(line_image, (int(x), int(y)), 2, (255, 0, 0), -1)

        if right_lines:
            right_points = np.array(right_lines)
            right_fit = np.polyfit(right_points[:, 1], right_points[:, 0], 1)
            right_x = right_fit[0] * plot_y + right_fit[1]
            for x, y in zip(right_x, plot_y):
                cv2.circle(line_image, (int(x), int(y)), 2, (0, 0, 255), -1)

    # ---------------------------------------------------------------
    # Obstacle Detection with color classification
    obstacle_crop_margin = int(0.35 * width)
    obstacle_roi = frame[:, obstacle_crop_margin:width - obstacle_crop_margin]
    hsv_obstacle = cv2.cvtColor(obstacle_roi, cv2.COLOR_BGR2HSV)

    color_ranges = [
        (np.array([0, 0, 0]), np.array([180, 255, 50]), 'Black'),
        (np.array([90, 50, 50]), np.array([130, 255, 255]), 'Blue'),
        (np.array([0, 70, 50]), np.array([10, 255, 255]), 'Red'),
        (np.array([160, 70, 50]), np.array([180, 255, 255]), 'Red')
    ]

    combined_mask = np.zeros_like(hsv_obstacle[:, :, 0])
    color_mask_map = []

    for lower, upper, color_name in color_ranges:
        mask = cv2.inRange(hsv_obstacle, lower, upper)
        color_mask_map.append((mask, color_name))
        combined_mask = cv2.bitwise_or(combined_mask, mask)

    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    obstacle_detected = False
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 500 and area < 750:
            x, y, w, h = cv2.boundingRect(cnt)
            center_x = x + w // 2
            x_global = x + obstacle_crop_margin
            center_x_global = center_x + obstacle_crop_margin

            if (width // 4 < center_x_global < 3 * width // 4):
                mask_roi = np.zeros_like(combined_mask)
                cv2.drawContours(mask_roi, [cnt], -1, 255, -1)
                mean_hsv = cv2.mean(hsv_obstacle, mask=mask_roi)[:3]

                detected_color_name = "Unknown"
                for lower, upper, color_name in color_ranges:
                    if np.all(lower <= mean_hsv) and np.all(mean_hsv <= upper):
                        detected_color_name = color_name
                        break

                obstacle_detected = True
                cv2.rectangle(frame, (x_global, y), (x_global + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, f"Obstacle: {detected_color_name}", (x_global, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.imshow('Obstacle ROI', obstacle_roi)

    # ---------------------------------------------------------------
    # Decision Making - Raw State
    if obstacle_detected:
        current_decision = "STOP - Obstacle Ahead"
    elif right_dashed and not left_dashed:
        current_decision = "Right Turn Possible – Dashed Line"
    elif left_dashed and not right_dashed:
        current_decision = "Left Turn Possible – Dashed Line"
    elif not left_lines and not right_lines:
        current_decision = "No Lane Detected"
    else:
        current_decision = "Go Straight"

    # Use state machine to stabilize the decision
    direction_text = lane_state_machine.update(current_decision)

    # ---------------------------------------------------------------
    # Final Display
    combined_lower = cv2.addWeighted(lower_half_cropped, 0.7, cv2.cvtColor(lane_mask, cv2.COLOR_GRAY2BGR), 0.3, 0)
    combined_lower = cv2.addWeighted(combined_lower, 0.9, cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 0.4, 0)

    combined_full = frame.copy()
    combined_full[height // 2:, crop_margin:width - crop_margin] = combined_lower
    output = cv2.addWeighted(combined_full, 1, line_image, 0.6, 0)

    cv2.rectangle(output, (20, 20), (620, 80), (0, 0, 0), -1)
    cv2.putText(output, direction_text, (30, 65), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

    frame_count += 1
    if frame_count >= 10:
        curr_time = time.time()
        fps = frame_count / (curr_time - prev_time)
        prev_time = curr_time
        frame_count = 0

    cv2.putText(output, f"FPS: {fps:.2f}", (700, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    print()
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Lane Area', line_image)
    cv2.imshow('Final Output', output)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
def detect_objects_on_road(frame):
    global previous_frame
    height, width = frame.shape[:2]
    roi_polygon = np.array([[(100, height), (2200, height), (1000, 260), (700, 260)]])
    roi_mask = np.zeros((height, width), dtype=np.uint8)
    cv2.fillPoly(roi_mask, roi_polygon, 255)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_and(gray, gray, mask=roi_mask)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    if previous_frame is None:
        previous_frame = blur
        return frame, False

    diff = cv2.absdiff(previous_frame, blur)
    _, thresh = cv2.threshold(diff, 60, 255, cv2.THRESH_BINARY)
    thresh = cv2.bitwise_and(thresh, thresh, mask=roi_mask)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    thresh = cv2.dilate(thresh, kernel, iterations=2)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = False
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 1000 or area > 20000:
            continue

        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = h / float(w + 1)

        if aspect_ratio < 1.5 or aspect_ratio > 3.0:
            continue

        cx, cy = x + w // 2, y + h // 2
        if roi_mask[cy, cx] == 255:
            box_mask = np.zeros_like(roi_mask)
            cv2.rectangle(box_mask, (x, y), (x + w, y + h), 255, -1)
            intersection = cv2.bitwise_and(box_mask, roi_mask)
            overlap_area = cv2.countNonZero(intersection)
            box_area = w * h

            if overlap_area / box_area > 0.7:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                detected = True

    previous_frame = blur
    return frame, detected

In [ ]:
import cv2
import numpy as np
import time
from collections import defaultdict

# -------------------- Lane State Machine Class --------------------

class LaneStateMachine:
    def _init_(self, stability_threshold=5):
        self.state = "No Lane Detected"
        self.counters = defaultdict(int)
        self.threshold = stability_threshold

    def update(self, new_state):
        for key in self.counters:
            if key != new_state:
                self.counters[key] = 0
        self.counters[new_state] += 1

        if self.counters[new_state] >= self.threshold:
            if new_state != self.state:
                self.state = new_state
                for key in self.counters:
                    self.counters[key] = 0
        return self.state

# -------------------- Utility Functions --------------------

def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    cl = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8)).apply(l)
    return cv2.cvtColor(cv2.merge((cl, a, b)), cv2.COLOR_LAB2BGR)

def detect_lanes(frame, crop_margin_percent):
    height, width = frame.shape[:2]
    crop_margin_x = int(width * crop_margin_percent)
    crop_margin_y = int(height * 0.5)  # Half of the height for the ROI

    roi = frame[crop_margin_y:, crop_margin_x:width - crop_margin_x]

    # Apply CLAHE for contrast enhancement
    clahe_roi = apply_clahe(roi)

    # HSV yellow mask with tightened thresholds to reduce grass
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    # Bright yellow filtering
    hsv_yellow_mask = cv2.inRange(hsv, np.array([20, 100, 100]), np.array([35, 255, 255]))
    v_mask = cv2.inRange(v, 150, 255)
    hsv_yellow_mask = cv2.bitwise_and(hsv_yellow_mask, hsv_yellow_mask, mask=v_mask)

    # LAB yellow mask
    lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB)
    lab_yellow_mask = cv2.inRange(lab, np.array([150, 120, 120]), np.array([255, 160, 160]))

    # Combine HSV + LAB masks
    combined_mask = cv2.bitwise_or(hsv_yellow_mask, lab_yellow_mask)

    # Morphological filtering
    kernel = np.ones((7, 7), np.uint8)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)

    # Edge detection
    masked_roi = cv2.bitwise_and(clahe_roi, clahe_roi, mask=combined_mask)
    gray = cv2.cvtColor(masked_roi, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 100, 200)
    edges = cv2.bitwise_and(edges, combined_mask)

    # Detect lines
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=30)
    return combined_mask, edges, lines, roi

def classify_lane_lines(lines, crop_margin_percent, height):
    left_lines, right_lines = [], []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            x1 += int(crop_margin_percent * 960)
            x2 += int(crop_margin_percent * 960)
            y1 += height // 2
            y2 += height // 2

            if x2 - x1 == 0:
                continue

            slope = (y2 - y1) / (x2 - x1)
            angle = np.degrees(np.arctan(abs(slope)))

            if 30 < angle < 80:  # Typical lane angles
                if slope < 0:
                    left_lines.extend([(x1, y1), (x2, y2)])
                else:
                    right_lines.extend([(x1, y1), (x2, y2)])
    return left_lines, right_lines

def shade_lane_region(frame, left_lines, right_lines):
    height = frame.shape[0]
    overlay = np.zeros_like(frame)

    if left_lines and right_lines:
        left_points = np.array(left_lines).reshape(-1, 2)
        right_points = np.array(right_lines).reshape(-1, 2)

        left_fit = np.polyfit(left_points[:, 1], left_points[:, 0], 1)
        right_fit = np.polyfit(right_points[:, 1], right_points[:, 0], 1)

        plot_y = np.linspace(height // 2, height - 1, num=height // 2)
        left_x = left_fit[0] * plot_y + left_fit[1]
        right_x = right_fit[0] * plot_y + right_fit[1]

        pts_left = np.array([np.transpose(np.vstack([left_x, plot_y]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_x, plot_y])))])
        pts = np.hstack((pts_left, pts_right)).astype(np.int32)

        cv2.fillPoly(overlay, [pts], (0, 255, 255))  # Yellow fill

    return overlay

# -------------------- Main Function --------------------

def main(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    lane_state_machine = LaneStateMachine(stability_threshold=5)
    crop_margin_percent = 0.2 # 15% of width for left/right cropping
    obstacle_crop_margin_percent = 0.35  # 35% for obstacle detection cropping

    prev_time, frame_count, fps = time.time(), 0, 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (960, 540))
        height, width = frame.shape[:2]

        # Lane Detection
        lane_mask, edges, lines, lane_roi = detect_lanes(frame, crop_margin_percent)
        left_lines, right_lines = classify_lane_lines(lines, crop_margin_percent, height)
        lane_overlay = shade_lane_region(frame, left_lines, right_lines)

        # Obstacle Detection (if required)
        obstacle_detected = False  # Remove this function if not needed

        # Decision Making
        if obstacle_detected:
            current_decision = "STOP - Obstacle Ahead"
        elif not left_lines and not right_lines:
            current_decision = "No Lane Detected"
        else:
            current_decision = "Go Straight"

        decision = lane_state_machine.update(current_decision)

        # Overlay display
        combined_lower = cv2.addWeighted(lane_roi, 0.7, cv2.cvtColor(lane_mask, cv2.COLOR_GRAY2BGR), 0.3, 0)
        combined_lower = cv2.addWeighted(combined_lower, 0.9, cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 0.4, 0)
        full_frame = frame.copy()
        full_frame[height // 2:, int(width * crop_margin_percent):width - int(width * crop_margin_percent)] = combined_lower
        output = cv2.addWeighted(full_frame, 1, lane_overlay, 0.6, 0)
        cv2.rectangle(output, (20, 20), (620, 80), (0, 0, 0), -1)
        cv2.putText(output, decision, (30, 65), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

        frame_count += 1
        if frame_count >= 10:
            curr_time = time.time()
            fps = frame_count / (curr_time - prev_time)
            prev_time = curr_time
            frame_count = 0

        cv2.putText(output, f"FPS: {fps:.2f}", (700, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Show windows
        cv2.imshow('Original Frame', frame)
        cv2.imshow('Lane Area', lane_overlay)
        cv2.imshow('Final Output', output)
        cv2.imshow("Canny Edge Output",edges)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "PXL_20250325_043922504.TS.mp4"

    main(video_path)

In [24]:
import cv2
import numpy as np
import time
from collections import defaultdict

# Path to your video file
video_path = "PXL_20250325_043754655.TS.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error opening video file")
    exit()

# For FPS calculation
prev_time = time.time()
frame_count = 0
fps = 0

# Initialize background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2()

# Lane detection parameters (keep these as in your code)
def is_dashed(lines, threshold=20):
    if len(lines) < 4:
        return False
    lines = sorted(lines, key=lambda p: p[1])  # sort by y
    gaps = []
    for i in range(0, len(lines) - 2, 2):
        y1 = lines[i][1]
        y2 = lines[i + 2][1]
        gaps.append(abs(y2 - y1))
    if len(gaps) == 0:
        return False
    avg_gap = sum(gaps) / len(gaps)
    return avg_gap > threshold

class LaneStateMachine:
    def __init__(self, stability_threshold=5):
        self.state = "No Lane Detected"
        self.counters = defaultdict(int)
        self.threshold = stability_threshold

    def update(self, new_state):
        for key in self.counters:
            if key != new_state:
                self.counters[key] = 0

        self.counters[new_state] += 1

        if self.counters[new_state] >= self.threshold:
            if new_state != self.state:
                self.state = new_state
                for key in self.counters:
                    self.counters[key] = 0
        return self.state

# Initialize the state machine
lane_state_machine = LaneStateMachine(stability_threshold=4)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (960, 540))
    height, width, _ = frame.shape

    # ---------------------------------------------------------------
    # Detect yellow lanes to define road area
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([15, 30, 30])  # Lower bound of yellow color
    upper_yellow = np.array([40, 255, 255])  # Upper bound of yellow color
    yellow_mask = cv2.inRange(hsv_frame, lower_yellow, upper_yellow)

    # Use morphological operations to clean up the yellow lane mask
    kernel = np.ones((5, 5), np.uint8)
    yellow_mask = cv2.morphologyEx(yellow_mask, cv2.MORPH_CLOSE, kernel)
    yellow_mask = cv2.morphologyEx(yellow_mask, cv2.MORPH_OPEN, kernel)

    # Find contours in the yellow lane mask to get lane boundaries
    contours, _ = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    lane_contours = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 500:  # Threshold to avoid small noise contours
            lane_contours.append(cnt)

    # Create a mask for the road area (only between yellow lanes)
    road_mask = np.zeros_like(frame[:, :, 0])  # Single channel mask
    cv2.drawContours(road_mask, lane_contours, -1, (255), thickness=cv2.FILLED)

    # Initialize a grey road area (start with black image)
    grey_road = np.zeros_like(frame)
    grey_road[:] = [0, 0, 0]  # Start with black color

    # Detect the first and second yellow lanes
    lane_positions = []
    for cnt in lane_contours:
        # Find the bounding box of each lane and record its horizontal position
        x, y, w, h = cv2.boundingRect(cnt)
        lane_positions.append(x)

    # Sort the lane positions from left to right
    lane_positions.sort()

    if len(lane_positions) >= 2:  # Check if both lanes are detected
        left_lane_x = lane_positions[0]
        right_lane_x = lane_positions[1]

        # Define the region between the yellow lanes
        grey_road[:, left_lane_x:right_lane_x] = [128, 128, 128]  # Fill the area between lanes with grey

    # ---------------------------------------------------------------
    # Object Detection using Background Subtraction
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    # Apply the road mask to limit detection to the road area (everything outside the road is black)
    fgmask = cv2.bitwise_and(fgmask, fgmask, mask=road_mask)

    # Find contours for objects detected on the road
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    obstacle_detected = False

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 500 and area < 10000:  # Threshold for object size
            x, y, w, h = cv2.boundingRect(cnt)
            center_x = x + w // 2
            x_global = x
            center_x_global = center_x

            # Only consider objects within the road area (between yellow lanes)
            if (width // 4 < center_x_global < 3 * width // 4):
                cv2.rectangle(frame, (x_global, y), (x_global + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, "Obstacle Detected", (x_global, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                obstacle_detected = True

    # ---------------------------------------------------------------
    # Decision Making - Raw State
    if obstacle_detected:
        current_decision = "STOP - Obstacle Ahead"
    else:
        current_decision = "Go Straight"

    # Use state machine to stabilize the decision
    direction_text = lane_state_machine.update(current_decision)

    # ---------------------------------------------------------------
    # Final Display
    output = frame.copy()

    # Show the direction decision on the output
    cv2.rectangle(output, (20, 20), (620, 80), (0, 0, 0), -1)
    cv2.putText(output, direction_text, (30, 65), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

    # Show FPS on the output
    frame_count += 1
    if frame_count >= 10:
        curr_time = time.time()
        fps = frame_count / (curr_time - prev_time)
        prev_time = curr_time
        frame_count = 0

    cv2.putText(output, f"FPS: {fps:.2f}", (700, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Display grey road area in Lane Area window
    cv2.imshow('Lane Area', grey_road)

    cv2.imshow('Original Frame', frame)
    cv2.imshow('Final Output', output)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
import time
from collections import defaultdict
# Path to your video file
video_path = "PXL_20250325_043754655.TS.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error opening video file")
    exit()

# For FPS calculation
prev_time = time.time()
frame_count = 0
fps = 0

def is_dashed(lines, threshold=20):
    if len(lines) < 4:
        return False
    lines = sorted(lines, key=lambda p: p[1])  # sort by y
    gaps = []
    for i in range(0, len(lines) - 2, 2):
        y1 = lines[i][1]
        y2 = lines[i + 2][1]
        gaps.append(abs(y2 - y1))
    if len(gaps) == 0:
        return False
    avg_gap = sum(gaps) / len(gaps)
    return avg_gap > threshold
from collections import defaultdict

class LaneStateMachine:
    def __init__(self, stability_threshold=5):
        self.state = "No Lane Detected"
        self.counters = defaultdict(int)
        self.threshold = stability_threshold

    def update(self, new_state):
        # Reset all counters except the current suggestion
        for key in self.counters:
            if key != new_state:
                self.counters[key] = 0

        self.counters[new_state] += 1

        # If new_state is consistent for N frames, update state
        if self.counters[new_state] >= self.threshold:
            if new_state != self.state:
                self.state = new_state
                # Reset all counters after transition
                for key in self.counters:
                    self.counters[key] = 0
        return self.state


In [15]:
import cv2
import numpy as np

# Open the video file
cap = cv2.VideoCapture('PXL_20250325_043754655.TS.mp4')

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error: Could not open video file")
    exit()

# Loop through each frame in the video
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to read frame")
        break

    frame = cv2.resize(frame, (960, 540))

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise and improve edge detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detection to find edges in the image

    edges = cv2.Canny(blurred, 100, 200)

    # Find contours from the edges
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate over the contours and filter out small ones
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter out small contours
            # Get the bounding box for each detected object
            x, y, w, h = cv2.boundingRect(contour)

            # Draw a rectangle around the detected object
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Optionally, show the Canny edge image and the original frame
    cv2.imshow('Edges', edges)
    cv2.imshow('Detected Objects', frame)

    # Check for key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


Error: Failed to read frame


In [4]:
import cv2
import numpy as np

# Open the video file
cap = cv2.VideoCapture('PXL_20250325_043754655.TS.mp4')

if not cap.isOpened():
    print("Error: Could not open video file")
    exit()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to read frame")
        break

    frame = cv2.resize(frame, (960, 540))

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Canny edge detection
    edges = cv2.Canny(blurred, 100, 200)

    # Define Region of Interest (ROI) for lane detection (focus on the road)
    height, width = edges.shape
    roi_vertices = [(0, height), (width // 2, height // 2), (width, height)]
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, np.array([roi_vertices], np.int32), 255)
    roi_edges = cv2.bitwise_and(edges, mask)

    # Apply Hough Transform to detect lanes
    lines = cv2.HoughLinesP(roi_edges, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=200)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Show the result
    cv2.imshow('Detected Lanes', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Error: Failed to read frame


In [20]:
import cv2
import numpy as np

def detect_lanes_and_objects(frame):
    # Resize for consistency
    frame = cv2.resize(frame, (960, 540))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Canny edge detection
    edges = cv2.Canny(blur, 50, 150)

    # Define region of interest (trapezoid shape for lane area)
    mask = np.zeros_like(edges)
    h, w = edges.shape
    roi_corners = np.array([[
        (w//10, h),
        (w//2 - 50, h//2 + 40),
        (w//2 + 50, h//2 + 40),
        (w - w//10, h)
    ]], dtype=np.int32)
    cv2.fillPoly(mask, roi_corners, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Hough Line Transform for lane detection
    lines = cv2.HoughLinesP(masked_edges, 1, np.pi/180, 50, minLineLength=50, maxLineGap=100)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 255), 3)

    # Create lane polygon for object-in-lane check
    lane_polygon = roi_corners[0]

    # Object detection using thresholding and CCA
    _, binary = cv2.threshold(blur, 200, 255, cv2.THRESH_BINARY_INV)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    # Connected Component Analysis
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closed)

    object_detected = False
    object_on_lane = False

    for i in range(1, num_labels):  # Skip background
        area = stats[i, cv2.CC_STAT_AREA]
        if area > 500:
            x = stats[i, cv2.CC_STAT_LEFT]
            y = stats[i, cv2.CC_STAT_TOP]
            w = stats[i, cv2.CC_STAT_WIDTH]
            h = stats[i, cv2.CC_STAT_HEIGHT]
            cx, cy = centroids[i]

            object_detected = True
            if cv2.pointPolygonTest(lane_polygon, (int(cx), int(cy)), False) >= 0:
                object_on_lane = True
                color = (0, 0, 255)  # Red for objects on lane
            else:
                color = (0, 255, 0)  # Green for objects off lane

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.circle(frame, (int(cx), int(cy)), 3, (255, 0, 0), -1)

    # Display detection result
    cv2.putText(frame, f'Object: {"Detected" if object_detected else "Not"}', (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    cv2.putText(frame, f'On Lane: {"Yes" if object_on_lane else "No"}', (20, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    return frame, masked_edges


# =======================
# Main Execution (example for video)
# =======================
if __name__ == "__main__":
    cap = cv2.VideoCapture("PXL_20250325_043754655.TS.mp4")  # Replace with your filename

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        result_frame, edges = detect_lanes_and_objects(frame)

        cv2.imshow("Edges", edges)
        cv2.imshow("Detected Lanes & Objects", result_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


KeyboardInterrupt: 

In [13]:
import cv2
import numpy as np

def sobel_filter(img):
    """Apply Sobel operator to calculate gradients in X and Y directions."""
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    grad_x = cv2.filter2D(img, -1, sobel_x)
    grad_y = cv2.filter2D(img, -1, sobel_y)

    return grad_x, grad_y

def non_maximum_suppression(grad_magnitude, grad_direction):
    """Apply non-maximum suppression to thin out edges."""
    rows, cols = grad_magnitude.shape
    output = np.zeros_like(grad_magnitude)

    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            # Determine edge direction and round to nearest 45 degrees
            angle = grad_direction[i, j]
            if angle < 0:
                angle += 180
            if (0 <= angle < 22.5) or (157.5 <= angle < 180):
                neighbor1 = grad_magnitude[i, j + 1]
                neighbor2 = grad_magnitude[i, j - 1]
            elif (22.5 <= angle < 67.5):
                neighbor1 = grad_magnitude[i + 1, j - 1]
                neighbor2 = grad_magnitude[i - 1, j + 1]
            elif (67.5 <= angle < 112.5):
                neighbor1 = grad_magnitude[i + 1, j]
                neighbor2 = grad_magnitude[i - 1, j]
            elif (112.5 <= angle < 157.5):
                neighbor1 = grad_magnitude[i - 1, j - 1]
                neighbor2 = grad_magnitude[i + 1, j + 1]

            # Suppress non-maximum pixels
            if grad_magnitude[i, j] >= neighbor1 and grad_magnitude[i, j] >= neighbor2:
                output[i, j] = grad_magnitude[i, j]
            else:
                output[i, j] = 0

    return output

def double_thresholding(img, low_threshold, high_threshold):
    """Apply double thresholding to classify pixels as strong, weak, or non-edges."""
    strong_edges = np.zeros_like(img)
    weak_edges = np.zeros_like(img)

    strong_edges[img > high_threshold] = 255
    weak_edges[(img >= low_threshold) & (img <= high_threshold)] = 255

    return strong_edges, weak_edges

def edge_tracking_by_hysteresis(strong_edges, weak_edges):
    """Track edges by hysteresis (connect weak edges to strong edges)."""
    rows, cols = strong_edges.shape
    final_edges = np.copy(strong_edges)

    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            if weak_edges[i, j] == 255:
                # Check 8 neighbors to connect weak edges to strong edges
                if np.any(strong_edges[i - 1:i + 2, j - 1:j + 2] == 255):
                    final_edges[i, j] = 255
                else:
                    final_edges[i, j] = 0

    return final_edges

def manual_canny_edge_detection(img, low_threshold=100, high_threshold=200):
    """Manual Canny edge detection implementation."""
    # Step 1: Apply Gaussian Blur
    blurred = cv2.GaussianBlur(img, (5, 5), 0)

    # Step 2: Calculate gradients using Sobel filters
    grad_x, grad_y = sobel_filter(blurred)

    # Step 3: Calculate gradient magnitude and direction
    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    grad_direction = np.arctan2(grad_y, grad_x) * 180 / np.pi

    # Step 4: Apply non-maximum suppression
    non_max_suppressed = non_maximum_suppression(grad_magnitude, grad_direction)

    # Step 5: Apply double thresholding
    strong_edges, weak_edges = double_thresholding(non_max_suppressed, low_threshold, high_threshold)

    # Step 6: Perform edge tracking by hysteresis
    final_edges = edge_tracking_by_hysteresis(strong_edges, weak_edges)

    return final_edges

# Open the video file
cap = cv2.VideoCapture('PXL_20250325_043754655.TS.mp4')

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error: Could not open video file")
    exit()

# Loop through each frame in the video
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to read frame")
        break

    # Resize frame for easier processing (optional)
    frame = cv2.resize(frame, (960, 540))

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply manual Canny edge detection
    edges = manual_canny_edge_detection(gray, low_threshold=100, high_threshold=200)

    # Ensure the edge image is in the correct format for contours (uint8, values 0 or 255)
    edges = np.uint8(edges)

    # Find contours from the edges
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate over the contours and filter out small ones
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter out small contours
            # Get the bounding box for each detected object
            x, y, w, h = cv2.boundingRect(contour)

            # Draw a rectangle around the detected object
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Optionally, show the edge image and the original frame
    cv2.imshow('Edges', edges)
    cv2.imshow('Detected Objects', frame)

    # Check for key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [15]:
import cv2
import numpy as np

# Load the input image
image = cv2.imread('Fig01.tif')  # Replace 'your_image.jpg' with the path to your image

# Resize the image for easier visualization (optional)
image = cv2.resize(image, (960, 540))

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply GaussianBlur to reduce noise and improve edge detection
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Detect edges using Canny edge detector
edges = cv2.Canny(blurred, 100, 200)

# Apply Hough Line Transform to detect straight lines
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=200)

# If lines are detected, draw them on the original image
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 3)  # Draw green lines

# Display the original image with highlighted lines
cv2.imshow('Detected Lines', image)

# Wait for a key press and close all windows
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np

# Load the pre-trained Haar Cascade classifier for detecting humans (body)
object_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')

# Open the video file
cap = cv2.VideoCapture('PXL_20250325_043754655.TS.mp4')  # Replace with your video file path

if not cap.isOpened():
    print("Error: Could not open video file")
    exit()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to read frame")
        break


    frame = cv2.resize(frame, (960, 540))


    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    objs = object_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in objs:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)  # Green rectangle around human



    cv2.imshow('Object Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np

# Open the video file (you can use 'your_video.mp4' or an image path)
cap = cv2.VideoCapture('PXL_20250325_043754655.TS.mp4')
# Check if the video was opened successfully
if not cap.isOpened():
    print("Error: Could not open video file")
    exit()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to read frame")
        break

    # Resize the frame for easier processing (optional)
    frame = cv2.resize(frame, (960, 540))

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and improve edge detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Step 1: Detect edges using Canny edge detector
    edges = cv2.Canny(blurred, 100, 200)

    # Step 2: Detect lines using Hough Line Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=200)

    # Step 3: Draw the detected lines
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)  # Green lines

    # Step 4: Detect circles using Hough Circle Transform
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=30, param1=50, param2=30, minRadius=10, maxRadius=100)

    # Step 5: Draw the detected circles
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(frame, (x, y), r, (0, 0, 255), 3)  # Red circles
            cv2.rectangle(frame, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)  # Small rectangle at the center

    # Display the processed frame with detected objects
    cv2.imshow('Detected Objects (Lines & Circles)', frame)

    # Wait for key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 